### Load libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.style.use("ggplot")

### Load questions

In [ ]:
df = pd.read_csv("combined_v1.csv").drop(columns="Unnamed: 0")
df["n_classes"] = df.classes.str.count(",") + 1
df["baseline_acc"] = 1 / df.n_classes
df["baseline_loss"] = -np.log(df.baseline_acc)
df

In [ ]:
df.n_classes.hist(bins=10);

### Load evaluation results

In [ ]:
CSV_MAP: dict[str, str] = {
    "ada": "combined_v1_gpt3/classification/ada.csv",
    "babbage": "combined_v1_gpt3/classification/babbage.csv",
    "curie": "combined_v1_gpt3/classification/curie.csv",
    "davinci": "combined_v1_gpt3/classification/davinci.csv",
    "ada_rlhf": "combined_v1_gpt3_rlhf/classification/text-ada-001.csv",
    "babbage_rlhf": "combined_v1_gpt3_rlhf/classification/text-babbage-001.csv",
    "curie_rlhf": "combined_v1_gpt3_rlhf/classification/text-curie-001.csv",
    "davinci_rlhf": "combined_v1_gpt3_rlhf/classification/text-davinci-001.csv",
    "opt_125m": "combined_v1_opt/classification/opt-125m.csv",
    "opt_350m": "combined_v1_opt/classification/opt-350m.csv",
    "opt_1_3b": "combined_v1_opt/classification/opt-1.3b.csv",
    "opt_2_7b": "combined_v1_opt/classification/opt-2.7b.csv",
}

In [ ]:
for k, path in CSV_MAP.items():
    cur_df = pd.read_csv("results/" + path)

    assert len(df) == len(cur_df)

    df[f"loss_{k}"] = cur_df.loss
    df[f"correct_{k}"] = cur_df.correct
    df[f"partial_credit_{k}"] = cur_df.partial_credit

df.columns.sort_values()


### Plotting code
[Link to model sizes](https://github.com/ed1d1a8d/inverse-scaling-eval-pipeline/blob/d7b0de68258525fddb2fb1f05091755ebaa24775/eval_pipeline/plot_loss.py#L19)

In [ ]:
GPT3_SIZES = {
    # GPT-3 sizes are based on https://blog.eleuther.ai/gpt3-model-sizes/
    "ada": 350_000_000,
    "babbage": 1_300_000_000,
    "curie": 6_700_000_000,
    "davinci": 175_000_000_000,
}

GPT3_RLHF_SIZES = {
    "ada_rlhf": 350_000_000,
    "babbage_rlhf": 1_300_000_000,
    "curie_rlhf": 6_700_000_000,
    "davinci_rlhf": 175_000_000_000,
}

OPT_SIZES = {
    # opt sizes from their names
    "opt_125m": 125_000_000,
    "opt_350m": 350_000_000,
    "opt_1_3b": 1_300_000_000,
    "opt_2_7b": 2_700_000_000,
}

OGPT_SIZES = {"opt_125m": 125_000_000} | GPT3_SIZES

In [ ]:
def gen_plots(
    df: pd.DataFrame,
    size_dict: dict[str, int],
    title: str = "",
):
    models = list(size_dict.keys())
    sizes = np.array(list(size_dict.values()))

    losses = np.array([df[f"loss_{m}"].mean() for m in models])
    loss_errs = np.array([df[f"loss_{m}"].std() for m in models]) / np.sqrt(len(df))
    accs = np.array([df[f"correct_{m}"].mean() for m in models])
    partial_accs = np.array([df[f"partial_credit_{m}"].mean() for m in models])

    plt.figure(figsize=(16, 3))
    plt.suptitle(title)
    plt.tight_layout()

    def common_plot_setup(baseline: float):
        plt.axhline(
            baseline,
            linestyle="--",
            color="black",
            zorder=-1,
            label="random baseline",
        )
        plt.xscale("log")
        # plt.xlabel("Model size")
        plt.xticks(sizes, models)
        plt.legend()

    # Loss
    plt.subplot(1, 3, 1)
    plt.errorbar(sizes, losses, loss_errs, label="loss")
    plt.yscale("log")
    common_plot_setup(df.baseline_loss.mean())

    # Acc
    plt.subplot(1, 3, 2)
    plt.plot(sizes, accs, label="acc.")
    plt.ylim(-0.02, 1.02)
    common_plot_setup(df.baseline_acc.mean())

    # Partial acc
    plt.subplot(1, 3, 3)
    plt.plot(sizes, partial_accs, label="partial acc.")
    plt.ylim(-0.02, 1.02)
    common_plot_setup(0.5)


gen_plots(df, OGPT_SIZES, title="opt-125m + gpt3 scaling")


In [ ]:
def comprehensive_plot(df: pd.DataFrame):
    gen_plots(df, OPT_SIZES, title="OPT Scaling")
    gen_plots(df, GPT3_SIZES, title="GPT3 Scaling")
    gen_plots(df, GPT3_RLHF_SIZES, title="GPT3 (rlhf) Scaling")

comprehensive_plot(df)

### Scaling laws by dataset
TODO